In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pyarrow
import random

from pathlib import Path
from config.paths import ROOT_DIR, SAMPLE_DIR, PRICES_DIR

import src.fileutils as files
import src.visualization as viz
import src.process as process

In [2]:
stations_info_file = ROOT_DIR / 'data' / 'stations.csv'
sample_file_location = SAMPLE_DIR
sample_price_location = SAMPLE_DIR / 'prices'

RSEED = 42
random.seed(RSEED)
np.random.seed(RSEED)

In [29]:
prices_df = pd.read_csv(files.pick_random_csv(PRICES_DIR, random_state=RSEED))
#prices_sample = pd.read_csv(PRICES_DIR / '2021' / '08' / '2021-08-09-prices.csv')



display(prices_df.sample(3, random_state=RSEED))
display(viz.nice_summary(prices_df))

,date,station_uuid,diesel,e5,e10,dieselchange,e5change,e10change
195420,2021-08-09 15:34:06+02,c83474cf-5736-43eb-adc5-91ccb28bb5e8,1.359,1.599,1.539,3,3,3
78104,2021-08-09 10:11:06+02,001e3111-58c3-4e99-b6d5-a8f244b579f3,1.449,1.659,1.599,0,1,1
269760,2021-08-09 18:34:08+02,9b7b1b32-4532-4e99-8b29-9edb64d44a0a,1.359,1.599,1.539,0,1,1


,Columns,Dtype,nunique,Non-Null Count,Missing,Missing %,Zero Count,mean,std,min,25%,50%,75%,max
0,date,object,1257,331244,-,-,0,-,-,-,-,-,-,-
1,station_uuid,object,14999,331244,-,-,0,-,-,-,-,-,-,-
2,diesel,float64,181,331244,-,-,33,1.39,0.05,-0.0,1.36,1.39,1.42,2.0
3,e5,float64,181,331244,-,-,4769,1.6,0.2,-0.0,1.59,1.62,1.65,2.07
4,e10,float64,176,331244,-,-,14572,1.49,0.32,-0.0,1.53,1.56,1.59,2.03
5,dieselchange,int64,4,331244,-,-,65371,0.89,0.61,0.0,1.0,1.0,1.0,3.0
6,e5change,int64,4,331244,-,-,67445,0.89,0.61,0.0,1.0,1.0,1.0,3.0
7,e10change,int64,4,331244,-,-,75399,0.86,0.61,0.0,1.0,1.0,1.0,3.0


In [47]:
def extend_panel(df, date='date', individual='station_uuid', names=['date','station']):

    df = process.set_panel_index(df, date=date, individual=individual)
    timestamps = process.get_unique_timestamps(df)
    stations = process.get_unique_index(df, individual)
    new_index = pd.MultiIndex.from_product([timestamps, stations], names=names)
    return df.reindex(new_index)

df = extend_panel(prices_df)
df

diesel     e5   
date                      station                                               
2021-08-09 00:01:07+02:00 00299b2b-4fcf-454a-9dfd-871a57bfe4ff   1.349  1.589  \
                          65b83b5f-f0c5-4db0-8be1-e111cd066276   1.349  1.589   
                          3e8b54ba-efd2-48ea-b948-7104f540d930   1.429  1.659   
                          dfc32f99-aa65-4fd4-b16e-a2cc53f9be15   1.429  1.669   
                          a1af2da1-8095-43ff-9640-7c0260c00048   1.419  1.609   
...                                                                ...    ...   
2021-08-09 23:59:08+02:00 998a6b7c-d1c9-4b3e-9c02-ac7196e5e9cb     NaN    NaN   
                          99eaeb13-893e-483e-9c26-2876d920a250     NaN    NaN   
                          a3c05496-1f6d-4d4a-a122-8cf4b464a0c1     NaN    NaN   
                          f2ef6fe5-9589-4b4e-8792-71f8b62f27c7     NaN    NaN   
                          0d86c9f2-3327-4bee-8ccc-a5bef904e9e6     NaN    NaN   

                                                                  e10   
date                      station                                       
2021-08-09 00:01:07+02:00 00299b2b-4fcf-454a-9dfd-871a57bfe4ff  1.529  \
                          65b83b5f-f0c5-4db0-8be1-e111cd066276  1.529   
                          3e8b54ba-efd2-48ea-b948-7104f540d930  1.599   
                          dfc32f99-aa65-4fd4-b16e-a2cc53f9be15  1.609   
                          a1af2da1-8095-43ff-9640-7c0260c00048  0.000   
...                                                               ...   
2021-08-09 23:59:08+02:00 998a6b7c-d1c9-4b3e-9c02-ac7196e5e9cb    NaN   
                          99eaeb13-893e-483e-9c26-2876d920a250    NaN   
                          a3c05496-1f6d-4d4a-a122-8cf4b464a0c1    NaN   
                          f2ef6fe5-9589-4b4e-8792-71f8b62f27c7    NaN   
                          0d86c9f2-3327-4bee-8ccc-a5bef904e9e6    NaN   

                                                                dieselchange   
date                      station                                              
2021-08-09 00:01:07+02:00 00299b2b-4fcf-454a-9dfd-871a57bfe4ff           1.0  \
                          65b83b5f-f0c5-4db0-8be1-e111cd066276           1.0   
                          3e8b54ba-efd2-48ea-b948-7104f540d930           1.0   
                          dfc32f99-aa65-4fd4-b16e-a2cc53f9be15           1.0   
                          a1af2da1-8095-43ff-9640-7c0260c00048           1.0   
...                                                                      ...   
2021-08-09 23:59:08+02:00 998a6b7c-d1c9-4b3e-9c02-ac7196e5e9cb           NaN   
                          99eaeb13-893e-483e-9c26-2876d920a250           NaN   
                          a3c05496-1f6d-4d4a-a122-8cf4b464a0c1           NaN   
                          f2ef6fe5-9589-4b4e-8792-71f8b62f27c7           NaN   
                          0d86c9f2-3327-4bee-8ccc-a5bef904e9e6           NaN   

                                                                e5change   
date                      station                                          
2021-08-09 00:01:07+02:00 00299b2b-4fcf-454a-9dfd-871a57bfe4ff       1.0  \
                          65b83b5f-f0c5-4db0-8be1-e111cd066276       1.0   
                          3e8b54ba-efd2-48ea-b948-7104f540d930       0.0   
                          dfc32f99-aa65-4fd4-b16e-a2cc53f9be15       1.0   
                          a1af2da1-8095-43ff-9640-7c0260c00048       0.0   
...                                                                  ...   
2021-08-09 23:59:08+02:00 998a6b7c-d1c9-4b3e-9c02-ac7196e5e9cb       NaN   
                          99eaeb13-893e-483e-9c26-2876d920a250       NaN   
                          a3c05496-1f6d-4d4a-a122-8cf4b464a0c1       NaN   
                          f2ef6fe5-9589-4b4e-8792-71f8b62f27c7       NaN   
                          0d86c9f2-3327-4bee-8ccc-a5bef904e9e6       NaN   

                              

In [48]:
df[['diesel', 'e5', 'e10']] = df.groupby(level='station')[['diesel', 'e5', 'e10']].fillna(method='ffill')


In [58]:
df = df.reset_index(level=1)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18853743 entries, 2021-08-09 00:01:07+02:00 to 2021-08-09 23:59:08+02:00
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   station       object 
 1   diesel        float64
 2   e5            float64
 3   e10           float64
 4   dieselchange  float64
 5   e5change      float64
 6   e10change     float64
dtypes: float64(6), object(1)
memory usage: 1.1+ GB


In [17]:
# active_stations = prices_data09.station_uuid.unique()
# active_stations_sample = np.random.choice(active_stations, size=100)
# pds = prices_df.query('station_uuid in @active_stations_sample')
# pds



# create a table that carries all stations for each hour of the day

# group by the hour of the day, take the average price if a station is occuring more than once during that time

# if a station occurs, check the *change columns if its a 2 or a 3, and if yes, check if the price is actually different from the previous hour of if prices have just been re-reported

# if prices changed then make a 1 in the price-changed-dummies

# if there are multiple occurences of the same station within one hour, check which prices changed and make en entry for the respective dummy

# if there are multiple occurences of the same station within one hour, for each of the 3 fuel prices, count how often it changed

# take a batch for each hour of the day

# check 

df = pd.merge(timestamps, stations, how='cross').set_index(['date', 'station'])
# df = pd.merge(df, pds, how='left', on=['date', 'station_uuid']).set_index(['date', 'station_uuid'])

df

Empty DataFrame
Columns: []
Index: [(2021-08-09 00:01:07+02:00, 00299b2b-4fcf-454a-9dfd-871a57bfe4ff), (2021-08-09 00:01:07+02:00, 65b83b5f-f0c5-4db0-8be1-e111cd066276), (2021-08-09 00:01:07+02:00, 3e8b54ba-efd2-48ea-b948-7104f540d930), (2021-08-09 00:01:07+02:00, dfc32f99-aa65-4fd4-b16e-a2cc53f9be15), (2021-08-09 00:01:07+02:00, a1af2da1-8095-43ff-9640-7c0260c00048), (2021-08-09 00:01:07+02:00, 4ffec7ae-f7c2-4da5-a2c4-5395dd03f833), (2021-08-09 00:01:07+02:00, 4c7b7a7f-d642-4bcd-8d29-eec72cbbd2a2), (2021-08-09 00:01:07+02:00, 15e3dded-c0f3-4fe0-8eff-f9ad6e28e6a8), (2021-08-09 00:01:07+02:00, 57f207da-80a0-4e61-b98d-d6eef0d79cfe), (2021-08-09 00:01:07+02:00, a7b0746c-8c29-42b6-a7a0-5abcc6c177ea), (2021-08-09 00:01:07+02:00, 87ac5c03-273a-4b5c-8caa-a9a74c2ff3f0), (2021-08-09 00:01:07+02:00, da05f8af-a6ad-4145-9661-b79fc42b2349), (2021-08-09 00:01:07+02:00, c2c6c9f8-41ee-435f-8cdc-870d923155ef), (2021-08-09 00:01:07+02:00, 79b29abc-593f-4b72-94f5-75188a943d86), (2021-08-09 00:01:07+02:00, b05210cd-c08e-44d6-a233-59916f0ac367), (2021-08-09 00:01:07+02:00, f8931c9a-1814-4fcb-84bc-4dcf2b2913b8), (2021-08-09 00:01:07+02:00, ef77aedf-2efb-48d6-a7af-543f460068e9), (2021-08-09 00:01:07+02:00, 47a417dc-a6f9-4890-b8e6-34672ab32f13), (2021-08-09 00:01:07+02:00, 3a7013d0-6566-4c9d-ab0a-3536a80d9828), (2021-08-09 00:01:07+02:00, dfe3bd39-83cd-43b9-8628-f09896331b19), (2021-08-09 00:01:07+02:00, 39af7edb-35b3-4b01-80d2-f455def5966d), (2021-08-09 00:01:07+02:00, 9876c291-7600-40db-9f39-84fdabbcd66e), (2021-08-09 00:01:07+02:00, 0cc777e4-13bc-48d4-b161-0f23d56afea6), (2021-08-09 00:01:07+02:00, 097e5a47-bd7c-4d68-9f9d-7529c347b696), (2021-08-09 00:01:07+02:00, f4c21018-060e-4d56-88c7-e7caede0d453), (2021-08-09 00:01:07+02:00, cf9bbafa-b256-4de9-8cd3-90156ef7d119), (2021-08-09 00:01:07+02:00, d1c105b5-968e-4af1-92e8-044c72897c46), (2021-08-09 00:01:07+02:00, c0d13adc-9c12-40c5-944d-e33aacc313a8), (2021-08-09 00:01:07+02:00, 6c278842-028e-47c1-b9e4-98a951dff90a), (2021-08-09 00:01:07+02:00, 92825bd9-b322-410c-8068-ac752051922b), (2021-08-09 00:01:07+02:00, a194baf4-a63a-4375-b97d-e2ec936133e5), (2021-08-09 00:01:07+02:00, bcef8473-305c-4617-aa90-a87a459786ee), (2021-08-09 00:01:07+02:00, 8fb359e2-de12-4137-b74c-729f5357cdaa), (2021-08-09 00:01:07+02:00, 92f5dbe7-2110-4e2d-a88c-23835cc904bf), (2021-08-09 00:01:07+02:00, e80169b2-1a75-457c-9e92-8da3d77a0f5d), (2021-08-09 00:01:07+02:00, c5335c8c-843a-48aa-a7fb-8bc4fc379ded), (2021-08-09 00:01:07+02:00, af5c4ab1-ba19-4ba2-8f8f-c63bfafdf834), (2021-08-09 00:01:07+02:00, e2b1503f-6be0-4f7b-979f-b105f3ab28c9), (2021-08-09 00:01:07+02:00, b47667a2-e7d6-40dc-bd0c-a45e37e7bc45), (2021-08-09 00:01:07+02:00, b84b16c1-2dad-4ed0-8823-916d8ed954bf), (2021-08-09 00:01:07+02:00, 8029a3e2-da9f-4e02-880f-95b7111db29e), (2021-08-09 00:01:07+02:00, 2cb1d93b-3937-402e-bb7f-7c96fc6fe1b7), (2021-08-09 00:01:07+02:00, 2b91e044-603e-450f-bcab-e7553a500ffd), (2021-08-09 00:01:07+02:00, c27bc772-66da-4781-af21-69dc58d6ea70), (2021-08-09 00:01:07+02:00, 9289d2c9-8d32-44ea-bf74-432d36c02425), (2021-08-09 00:01:07+02:00, c7b1d8f5-30d8-453c-b779-f0ffc51c8100), (2021-08-09 00:01:07+02:00, 44912b09-2184-44b9-8310-2a1f80f0d3e2), (2021-08-09 00:01:07+02:00, 42ddd4e2-9c2e-41d5-b441-eb9f54259535), (2021-08-09 00:01:07+02:00, 4a560453-e93f-40e9-9d05-4df1408aafc5), (2021-08-09 00:01:07+02:00, f26d47a1-ae37-4bff-a6fa-4a777a0eb425), (2021-08-09 00:01:07+02:00, a0dedaf4-e261-4e44-90b1-93f945be1483), (2021-08-09 00:01:07+02:00, 00060711-0001-4444-8888-acdc00000001), (2021-08-09 00:01:07+02:00, 8b2f8add-d210-46be-ae06-143ddd4cfa08), (2021-08-09 00:01:07+02:00, 3deeb08d-43d6-4b0e-9095-619cb3102649), (2021-08-09 00:01:07+02:00, fee0214f-76c7-414a-b38b-3acffe24628f), (2021-08-09 00:01:07+02:00, 073b06fc-e3d8-4955-80a1-f3f0f56fb146), (2021-08-09 00:01:07+02:00, 880fba77-0f8a-46af-a928-e69b8e4a8d37), (2021-08-09 00:01:07+02:00, 01124b8b-5e36-4a49-9906-7f96ad7d7db5), (2021-08-09 00:01:07+02:00, 00062573-533e-75a1-87f9-8a9f00062573), (2021-08-09

In [20]:

# create a MultiIndex with all combinations of timestamps and stations
new_index = pd.MultiIndex.from_product([timestamps, stations], names=['date', 'station'])

# reindex your original dataframe
prices_df = prices_df.reindex(new_index)

prices_df.head(10)


diesel     e5   
date                      station                                               
2021-08-09 00:01:07+02:00 00299b2b-4fcf-454a-9dfd-871a57bfe4ff   1.349  1.589  \
                          65b83b5f-f0c5-4db0-8be1-e111cd066276   1.349  1.589   
                          3e8b54ba-efd2-48ea-b948-7104f540d930   1.429  1.659   
                          dfc32f99-aa65-4fd4-b16e-a2cc53f9be15   1.429  1.669   
                          a1af2da1-8095-43ff-9640-7c0260c00048   1.419  1.609   
                          4ffec7ae-f7c2-4da5-a2c4-5395dd03f833   1.389  1.619   
                          4c7b7a7f-d642-4bcd-8d29-eec72cbbd2a2     NaN    NaN   
                          15e3dded-c0f3-4fe0-8eff-f9ad6e28e6a8     NaN    NaN   
                          57f207da-80a0-4e61-b98d-d6eef0d79cfe     NaN    NaN   
                          a7b0746c-8c29-42b6-a7a0-5abcc6c177ea     NaN    NaN   

                                                                  e10   
date                      station                                       
2021-08-09 00:01:07+02:00 00299b2b-4fcf-454a-9dfd-871a57bfe4ff  1.529  \
                          65b83b5f-f0c5-4db0-8be1-e111cd066276  1.529   
                          3e8b54ba-efd2-48ea-b948-7104f540d930  1.599   
                          dfc32f99-aa65-4fd4-b16e-a2cc53f9be15  1.609   
                          a1af2da1-8095-43ff-9640-7c0260c00048  0.000   
                          4ffec7ae-f7c2-4da5-a2c4-5395dd03f833  1.559   
                          4c7b7a7f-d642-4bcd-8d29-eec72cbbd2a2    NaN   
                          15e3dded-c0f3-4fe0-8eff-f9ad6e28e6a8    NaN   
                          57f207da-80a0-4e61-b98d-d6eef0d79cfe    NaN   
                          a7b0746c-8c29-42b6-a7a0-5abcc6c177ea    NaN   

                                                                dieselchange   
date                      station                                              
2021-08-09 00:01:07+02:00 00299b2b-4fcf-454a-9dfd-871a57bfe4ff           1.0  \
                          65b83b5f-f0c5-4db0-8be1-e111cd066276           1.0   
                          3e8b54ba-efd2-48ea-b948-7104f540d930           1.0   
                          dfc32f99-aa65-4fd4-b16e-a2cc53f9be15           1.0   
                          a1af2da1-8095-43ff-9640-7c0260c00048           1.0   
                          4ffec7ae-f7c2-4da5-a2c4-5395dd03f833           1.0   
                          4c7b7a7f-d642-4bcd-8d29-eec72cbbd2a2           NaN   
                          15e3dded-c0f3-4fe0-8eff-f9ad6e28e6a8           NaN   
                          57f207da-80a0-4e61-b98d-d6eef0d79cfe           NaN   
                          a7b0746c-8c29-42b6-a7a0-5abcc6c177ea           NaN   

                                                                e5change   
date                      station                                          
2021-08-09 00:01:07+02:00 00299b2b-4fcf-454a-9dfd-871a57bfe4ff       1.0  \
                          65b83b5f-f0c5-4db0-8be1-e111cd066276       1.0   
                          3e8b54ba-efd2-48ea-b948-7104f540d930       0.0   
                          dfc32f99-aa65-4fd4-b16e-a2cc53f9be15       1.0   
                          a1af2da1-8095-43ff-9640-7c0260c00048       0.0   
                          4ffec7ae-f7c2-4da5-a2c4-5395dd03f833       1.0   
                          4c7b7a7f-d642-4bcd-8d29-eec72cbbd2a2       NaN   
                          15e3dded-c0f3-4fe0-8eff-f9ad6e28e6a8       NaN   
                          57f207da-80a0-4e61-b98d-d6eef0d79cfe       NaN   
                          a7b0746c-8c29-42b6-a7a0-5abcc6c177ea       NaN   

                                                                e10change  
date                      station                                          
2021-08-09 00:01:07+02:00 00299b2b-4fcf-454a-9dfd-871a57bfe4ff        1.0  
                          65b83b5f-f0c5-4db0-8be1-e111cd066276        1.0  
                          3e8b54ba-e

In [ ]:
multi_index = pd.MultiIndex.from_product([timestamps, stations], names=['Date', 'Station'])
# df = pd.merge(df, pds, how='left', on=['date', 'station_uuid']).set_index(['date', 'station_uuid'])
df = pd.DataFrame(index=multi_index)

df